In [1]:
## https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

## https://github.com/aboSamoor/polyglot/issues/80
# conda install -c conda-forge pyicu morfessor icu -y && pip install pycld2 polyglot


# polyglot download TASK:ner2
# polyglot download TASK:embeddings2

In [2]:
data_path='../data/'
data_raw_path="../data/raw/"
data_man_path="../data/manipulate/"
model_path="../model/modeling1/"
model_path3="../model/modeling3/"
w2v_path="../data/w2v/"

In [3]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
import seaborn as sns
import nltk
from string import digits
import ast

import polyglot
from polyglot.text import Text, Word

from gensim.models import Word2Vec
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors

In [4]:
stop_words=open(data_path+"stopwords_id.txt")
stop_words=set([i[:-1] for i in stop_words.readlines()])

# Get The Data

In [5]:
# dtext=pd.read_csv(data_raw_path+"data_text_goshop_2.csv")
# dtag=pd.read_csv(data_raw_path+"data_tag_goshop.csv")

dtext2=pd.read_csv(data_raw_path+"data_raw_path_w_label.csv").dropna()
dtext2["real_tags"]=dtext2.real_tags.apply(lambda x : set(ast.literal_eval(x)))
dtag=pd.read_csv(data_raw_path+"data_tag_goshop.csv")

In [6]:
dtext2.head()

,order_no,item_list_cln2,real_tags,num_tags,num_text
0,441257845,nescafe kaleng latte dingin kaleng,{},0,5
1,145918103,allupurinol tablet mg strip celebrex tablet ta...,{tablet},1,11
2,391185000,telur butir,{telur},1,2
3,286334848,pack rokok sampoerna mild merah,"{sampoerna mild, rokok, mild merah, rokok samp...",4,5
4,219306620,bubur ayam dada cakue,"{cakue, ayam dada, ayam, bubur, bubur ayam}",5,4


## Our Embedding

In [7]:
embd2=Word2Vec.load(w2v_path+'our_vocab.bin')

In [13]:
embd2.wv.most_similar(positive="epson",topn=10)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('printer', 0.7852568626403809),
 ('tinta', 0.7743543386459351),
 ('cyan', 0.7576143145561218),
 ('canon', 0.7005780935287476),
 ('brother', 0.6984161138534546),
 ('deskjet', 0.6867681741714478),
 ('catridge', 0.6823409795761108),
 ('magenta', 0.6822640895843506),
 ('advantage', 0.6803412437438965),
 ('cartridge', 0.657386302947998)]

In [380]:
embd2.wv.most_similar(positive="loenpia",topn=20)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('lunpia', 0.8905543088912964),
 ('lumpiah', 0.6924533843994141),
 ('popiah', 0.6625952124595642),
 ('lumpia', 0.6511636972427368),
 ('popia', 0.6079772710800171),
 ('tisyu', 0.5513123869895935),
 ('pokpia', 0.5426343679428101),
 ('jalangkote', 0.541053295135498),
 ('jalankote', 0.539696216583252),
 ('baxo', 0.5264574289321899),
 ('kwotie', 0.5192034244537354),
 ('chaikue', 0.5129139423370361),
 ('pecindilan', 0.5032013654708862),
 ('kuotie', 0.4915176033973694),
 ('choipan', 0.4849822223186493),
 ('uyen', 0.4745851457118988),
 ('wotie', 0.4666287302970886),
 ('porsipangsit', 0.44786137342453003),
 ('kiau', 0.4421067237854004),
 ('coipan', 0.4402465522289276)]

In [23]:
dtext2[dtext2.item_list_cln2.str.contains(",")]

,order_no,item_list_cln2,real_tags,num_tags,num_text


In [ ]:
## Adjective and Noun
## Asinan-Rujak-Salad is food

## Problem
# 1. "Krim" and "Es Krim" <- solved on labeling
# 2. "Anggur" and "Anggur Merah" and other "fruit-beverage" problem <- solved on labeling
# 3. "ikan koi" and "koi" brand <- solved on labeling
# 4. "sisir" as noun and "sisir" as unit <- solved by remove sisir for temporary
# 5. "tepung" groceries and food <- solved on labeling
# 6. "Saos" groceries and food ?? <- removed
# 7. syrup beverage, groceries or medicine? <- removed
# 8. some animal is sign as "food" and the other sign as "animal-stuff"
# 9. "Pembersih" is household or beauty??
# 10. delete "acne"
# 11. "AC", "Bensin", "printer" means the accesories for those <- try to make it clearer
# 12. "karton" from stationary or food packaging?? (change to "kertas karton"), like "gelas" or "botol" household or food packaging (removed)
# 13. "kursi" is deleted because correlated with movies
# 14. 1000 over 1 million order is to get customer laundry
# 15. "papan" <- temporary removed 
# 16. "rumput" mostly for snack "tae kae noi"
# 17. "sendok" is to remind driver to bring one for the customer food
# 18. "kipas" electronic? or the form of food? <- change to "kipas angin"

### Exploration : Counting multi tags and text words 

In [14]:
tags_set=set(dtag.word.unique())
tags_set1=set([t for t in tags_set if len(t.split(" "))==1])
tags_set2=set([t for t in tags_set if len(t.split(" "))==2])

ts=set(dtag.word_tag.unique())
ts1=set([t for t in ts if len(t.split(" "))==1])
ts2=set([t for t in ts if len(t.split(" "))==2])

In [15]:
dict_word=pd.DataFrame(dtag.word_tag)
dict_word.index=dtag.word
dict_word=dict_word["word_tag"].to_dict()

dict_class=pd.DataFrame(dtag.classification)
dict_class.index=dtag.word_tag
dict_class=dict_class["classification"].to_dict()

In [16]:
dtext2[dtext2.num_tags==0].sample(100).head(10)

,order_no,item_list_cln2,real_tags,num_tags,num_text
138596,209072278,midi pleated dress warna hitam bust,{},0,6
1128708,482429236,sleepingbag matras pesanan atas nama indah,{},0,6
452170,425951822,jangkrik ons sama ulat jerman ons,{},0,6
1039680,138177769,lpg hijau kg tabungnya,{},0,4
473932,304136869,tali rapia gulung warna,{},0,4
66135,483990234,plat nkl qty pcs,{},0,4
921747,476479848,pax ibu profen mg pax cortiden dexamenthasol,{},0,7
43037,397541017,tirai magnet,{},0,2
945089,454347514,rati ara bakery rotidibayar go pay rb jadi say...,{},0,13
858996,477376613,ambil makanan atas nama nara,{},0,5


In [17]:
def check_tag2(list_X,i):
    if i in list_X:
        return 1
    else :
        return 0

In [18]:
len(tags_set),len(ts)

(452, 403)

Between 1 word

In [20]:
dtext2_samp1=dtext2.sample(150000)
for c in ts1:
    cek_tag=dtext2_samp1.real_tags.apply(lambda x : check_tag2(x,c))
    dtext2_samp1[c]=cek_tag

In [21]:
tag_corr=dtext2_samp1[list(ts1)].corr().abs()
list_corr = tag_corr.unstack()
so_1 = list_corr.sort_values(kind="quicksort",ascending=False)
so_1=so_1.reset_index()
so_1_g=so_1[so_1.level_0!=so_1.level_1].iloc[[i for i in range(40) if np.mod(i,2)==0]]

In [22]:
so_1_g

,level_0,level_1,0
243,printer,tinta,0.638677
245,tiket,film,0.490495
247,durian,pancake,0.334174
249,greentea,teh,0.328213
251,lele,pecel,0.290698
253,sate,kambing,0.285972
255,kursi,tiket,0.281717
257,rujak,pindang,0.277793
259,nasi,ayam,0.261692
261,lontong,sate,0.251454


In [23]:
dtext2_samp1[(dtext2_samp1["printer"]==1)&(dtext2_samp1["tinta"]==0)].head(10)
# dtext2_samp1[dtext2_samp1.order_no==462498455]["item_list_cln2"].tolist()

,order_no,item_list_cln2,real_tags,num_tags,num_text,teh,pembalut,tinta,salmon,bedak,...,kabel,sop,sepatu,pentol,kangkung,ovaltine,brownis,kopi,jaket,kerupuk
271978,479254274,kabel printer di rumah,"{kabel, printer}",2,4,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
361297,355736451,kabel usb printer canon,"{usb, kabel, printer}",3,4,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
454972,476902066,kertas thermal dus isi gulung untuk printer zj...,"{kertas, printer}",2,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1195649,322064890,printer,{printer},1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1020414,481746071,cartridge printer hp warna type,"{hp, printer}",2,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267243,431355875,cartridge printer hp warna hitam no,"{hp, printer}",2,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
958006,428379766,toner printer hp deskjet hitam color,"{hp, printer}",2,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
448310,407663707,catrige printer warna hitam untuk printer hp d...,"{kertas, hp, printer}",3,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
393048,483765299,cartridge printer canon pg,{printer},1,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
324228,484612101,printer blotooth,{printer},1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
## Conclusion
# 1. mostly "printer" means the accesorries of printer "cartridge" and hp (which is not Handphone)
# 2. Remove hp for now because its a "brand"
# 3. Use the embedding word that has been trained to see the synonym or more connected word

Between 2 word

In [25]:
dtext2_samp2=dtext2.sample(150000)

In [26]:
for c in ts2:
    cek_tag=dtext2_samp2.real_tags.apply(lambda x : check_tag2(x,c))
    dtext2_samp2[c]=cek_tag

In [27]:
tag_corr=dtext2_samp2[list(ts2)].corr().abs()
list_corr = tag_corr.unstack()
so_2 = list_corr.sort_values(kind="quicksort",ascending=False)

In [28]:
so_2=so_2.reset_index()
so_2_g=so_2[so_2.level_0!=so_2.level_1].iloc[[i for i in range(40) if np.mod(i,2)==0]]

In [29]:
so_2_g

,level_0,level_1,0
160,kuah pindang,rujak kuah,0.865789
162,es kopi,kopi susu,0.760788
164,ice blast,marlboro ice,0.568432
166,es teh,teh manis,0.556433
168,mie iblis,udang keju,0.490763
170,rokok sampoerna,sampoerna mild,0.463553
172,sampoerna mild,mild merah,0.444697
174,gula pasir,kuah pindang,0.443369
176,gula pasir,rujak kuah,0.427545
178,mie goreng,indomie goreng,0.422903


In [30]:
## Conclusion
# 1. merge "rujak kuah" and "kuah pindang" become "rujak kuah pindang" (from bali)
# 2. Remove "es kopi", "es teh"
# 3. Many Brand is catched this time which should be removed even before the tags is produced like "marlboro ice blast", "sampoerna mild merah", 
#  "sampoerna mild", "sari roti", "marlboro ice"  (but let "indomie" in sentence but remove from tags)
# 4. Use the embedding word that has been trained to see the synonym or more connected word
# 5. indomie -> labeled as mie

Between all word

In [31]:
dtext2_samp3=dtext2.sample(350000)

In [32]:
for c in ts:
    cek_tag=dtext2_samp3.real_tags.apply(lambda x : check_tag2(x,c))
    dtext2_samp3[c]=cek_tag

In [33]:
tag_corr=dtext2_samp3[list(ts)].corr().abs()
list_corr = tag_corr.unstack()
so_3 = list_corr.sort_values(kind="quicksort",ascending=False)

In [34]:
so_3=so_3.reset_index()
so_3_g=so_3[so_3.level_0!=so_3.level_1].iloc[[i for i in range(40) if np.mod(i,2)==0]]

In [35]:
so_3_g

,level_0,level_1,0
403,nasi uduk,uduk,0.983839
405,kuah pindang,rujak kuah,0.855496
407,anggur merah,anggur,0.821090
409,bubur,bubur ayam,0.800638
411,fries,french fries,0.797562
413,krim pemutih,pemutih,0.760991
415,bir bintang,bir,0.756130
417,es kopi,kopi susu,0.753982
419,indomie,indomie goreng,0.704458
421,nasi gudeg,gudeg,0.697023


In [36]:
## Conclusion
# 1. keep "uduk", remove "nasi uduk"
# 2. keep "gudeg", remove "nasi gudeg"
# 3. Remove "es kopi", "es teh"
# 4. Many Brand is catched this time which should be removed even before the tags is produced like "marlboro ice blast", "sampoerna mild merah", 
#  "sampoerna mild", "sari roti", "marlboro ice"  (but let "indomie" in sentence but remove from tags)
# 5. Use the embedding word that has been trained to see the synonym or more connected word
# 6. indomie -> labeled as mie
# 7. anggur merah -> alcohol
# 8. fries -> french fries

In [308]:
# check_L=list_corr.sort_values(kind="quicksort",ascending=False).reset_index()
# check_L[(check_L.level_0=="telur ayam")]

### find Synonym or typo

In [ ]:
# https://docs.google.com/spreadsheets/d/1hg4KKKdivoh6mG7Bena0jYdJSvzgABlWLrU0NX_vHTQ/edit#gid=34046762

In [356]:
from Levenshtein import distance,ratio

In [357]:
## get the keyword
dKey=pd.read_csv(data_man_path+"based_keyword.csv").drop_duplicates()
dKey.Keyword=dKey.Keyword.replace("losion","lotion")

In [358]:
# dKey.Keyword.tolist()
dKey2=pd.DataFrame()

In [359]:
## Get similarity score from embedding above 0.55 (if the number of similar word less than 15, get top 10)

In [360]:
kw="buku"
dKey2=pd.DataFrame()
for kw in dKey.Keyword.tolist():
    get_sw=pd.DataFrame(embd2.wv.most_similar(positive=kw,topn=30),columns=["Words","Similarity_Score"])
    len_above=len(get_sw[get_sw.Similarity_Score>=0.6])
    if len_above<=15:
        get_sw2=get_sw.head(10)
    else:
        get_sw2=get_sw[get_sw.Similarity_Score>=0.6]

    get_sw2["L_ratio"]=get_sw2.Words.apply(lambda x: ratio(x,kw))
    get_sw2["Final_Score"]=(0.6*get_sw2["L_ratio"])+(0.4*get_sw2.Similarity_Score)
    get_sw2["is_typo"]=(get_sw2.Final_Score>0.5).astype(int)
    get_sw2["Keywords"]=kw
    dKey2=dKey2.append(get_sw2)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

In [361]:
dKey2.shape

(3492, 6)

In [363]:
dKey2.columns=["Keywords","Similarity_Score","L_ratio","Final_Score","is_typo","Tags"]

In [365]:
# dKey2

In [367]:
dKey2.to_csv(data_man_path+"based_keyword_with_synonym_and_typo.csv",index=False)